In [1]:
import os
from typing import Tuple

from datetime import datetime, timedelta

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.json_loader import JSONLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
#from langchain_community.vectorstores import TimescaleVector
from langchain_community.vectorstores import TimescaleVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
huggingfacehub_api_token = os.environ['HUGGINGFACE_API_KEY']

In [ ]:
# Load the text and split it into chunks
loader = TextLoader("../data/reformatted_transcripts/2021-05-28 09.14.38_rf.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

In [ ]:
# Timescale Vector needs the service url to your cloud database. You can see this as soon as you create the
SERVICE_URL = os.environ["timescale_url"]

## 1. Similarity Search with Euclidean Distance (Default)

In [ ]:
# The TimescaleVector Module will create a table with the name of the collection.
COLLECTION_NAME = "testing"

# Create a Timescale Vector instance from the collection of documents
db = TimescaleVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    service_url=SERVICE_URL
)

In [ ]:
query = "What is microphenomenology?"
docs_with_score = db.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

In [ ]:
retriever = db.as_retriever()

In [ ]:
print(retriever)

## 3. Using ANN Search Indexes to Speed Up Queries

In [ ]:
# Initialize an existing TimescaleVector store
COLLECTION_NAME = "timescale_commits"

db = TimescaleVector(
    collection_name=COLLECTION_NAME,
    service_url=SERVICE_URL,
    embedding=embeddings,
)

In [ ]:
db.create_index()

In [ ]:
# drop the old index
db.drop_index()

# Create an HNSW index
# Note: You don't need to specify m and ef_construction parameters as we set smart defaults.
db.create_index(index_type="hnsw", m=16, ef_construction=64)

# rag timescale hybrid search time

speaker_name, zoom_time, transcript_content
name, YYYY-MM-DDTHH:MM:SS, text

In [3]:
SERVICE_URL = os.environ["timescale_url"]

In [4]:
collection_name = "eer_meetings"
partition_interval = timedelta(days=7)

In [5]:
document_content_description = "The transcript of EER meetings containing, speaker name, the date of the meeting and transcript content."
metadata_field_info = [
    AttributeInfo(
        name="file_id",
        description="name of the file",
        type="uuid",
    ),
    AttributeInfo(
        name="zoom_date",
        description="The timestamp of the file. Specify in YYYY-MM-DDTHH:MM:SS format",
        type="datetime.datetime",
    ),
    AttributeInfo(
        name="speaker_name",
        description="The name of the speaker of the content",
        type="string",
    ),
]

In [6]:
embeddings = HuggingFaceEmbeddings()

C:\Users\szh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from timescale_vector import client

ModuleNotFoundError: No module named 'timescale_vector'

In [8]:
vectorstore = TimescaleVector(
    embedding = embeddings,
    collection_name = collection_name,
    service_url = SERVICE_URL,
    time_partition_interval = partition_interval,
)

ImportError: Could not import timescale_vector python package. Please install it with `pip install timescale-vector`.